<a href="https://colab.research.google.com/github/stefarine/DMML2022_ROLEX/blob/main/code/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PREP METHODES

In [ ]:
!python -m spacy download fr_core_news_sm

In [2]:
import gensim
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import bs4 as bs
import urllib.request
import spacy
import string
import math
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import fr_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [4]:
#imports
def get_wordTypes(sentence):
  sp = spacy.load('fr_core_news_sm')
  my_doc = sp(sentence)
  mylist = []
  for word in my_doc:
    mylist.append(word.pos_)

  return mylist

def add_occurences_collumns(myDf):
  for i in range(myDf.shape[0]):
    rowIndex = myDf.index[i]
    for sentence in myDf.loc[rowIndex]:
      if type(sentence)==str:
        nbOccurence = Counter(get_wordTypes(sentence))
        #print(nbOccurence)
        for key in nbOccurence.keys():
          myDf.loc[rowIndex, key] = nbOccurence[key]

#il faut myDf.fillna(0) après

In [ ]:
def get_tfidfs_collumns(myX):
  tfidf = TfidfVectorizer(ngram_range=(1, 1))
  features = tfidf.fit_transform(myX)
  result_df = pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names()
    )
  return result_df

#PREP DATA

In [5]:
df = pd.read_csv('training_data.csv')
df_pred = pd.read_csv('unlabelled_test_data.csv')

In [48]:
X_pred = df_pred['sentence']
X_pred

0       Nous dûmes nous excuser des propos que nous eû...
1       Vous ne pouvez pas savoir le plaisir que j'ai ...
2       Et, paradoxalement, boire froid n'est pas la b...
3       Ce n'est pas étonnant, car c'est une saison my...
4       Le corps de Golo lui-même, d'une essence aussi...
                              ...                        
1195    C'est un phénomène qui trouve une accélération...
1196    Je vais parler au serveur et voir si on peut d...
1197    Il n'était pas comme tant de gens qui par pare...
1198        Ils deviennent dangereux pour notre économie.
1199    Son succès a généré beaucoup de réactions néga...
Name: sentence, Length: 1200, dtype: object

In [7]:
y = df['difficulty']
X = df['sentence']

In [8]:
X

0       Les coûts kilométriques réels peuvent diverger...
1       Le bleu, c'est ma couleur préférée mais je n'a...
2       Le test de niveau en français est sur le site ...
3                Est-ce que ton mari est aussi de Boston?
4       Dans les écoles de commerce, dans les couloirs...
                              ...                        
4795    C'est pourquoi, il décida de remplacer les hab...
4796    Il avait une de ces pâleurs splendides qui don...
4797    Et le premier samedi de chaque mois, venez ren...
4798    Les coûts liés à la journalisation n'étant pas...
4799    Sur le sable, la mer haletait de toute la resp...
Name: sentence, Length: 4800, dtype: object

#TEST

In [ ]:
myX = X[0:6]

In [ ]:
def get_df_with_tfidf_and_occurences(mySeries):
  myX_occurence = pd.DataFrame(mySeries)
  add_occurences_collumns(myX_occurence)
  myX_occurence = myX_occurence.fillna(0)
  myX_occurence = myX_occurence.drop(['sentence'],axis=1)

  #myX_tftruc = get_tfidfs_collumns(myX)
  #myDf_final = pd.concat([myX_tftruc, myX_occurence], axis=1)

  #Cette methode va retourner le nombre d'occurence de chaque type de mot sur le set de test qui est labelisé
  return myX_occurence

In [ ]:
#X = get_df_with_tfidf_and_occurences(X)

In [ ]:
X

,DET,NOUN,ADJ,VERB,ADP,PUNCT,CCONJ,ADV,PRON,AUX,SCONJ,PROPN,NUM,X,SYM,SPACE,INTJ
0,3.0,14.0,3.0,5.0,13.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,3.0,0.0,2.0,0.0,2.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,5.0,1.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4.0,10.0,2.0,4.0,11.0,5.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,4.0,4.0,3.0,6.0,4.0,3.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4796,4.0,6.0,0.0,3.0,5.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4797,3.0,4.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4798,7.0,11.0,3.0,2.0,3.0,2.0,2.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
X = pd.read_csv('occurences_X.csv')

In [16]:
X

,DET,NOUN,ADJ,VERB,ADP,PUNCT,CCONJ,ADV,PRON,AUX,SCONJ,PROPN,NUM,X,SYM,SPACE,INTJ
0,3.0,14.0,3.0,5.0,13.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,3.0,0.0,2.0,0.0,2.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,5.0,1.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4.0,10.0,2.0,4.0,11.0,5.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,4.0,4.0,3.0,6.0,4.0,3.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4796,4.0,6.0,0.0,3.0,5.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4797,3.0,4.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4798,7.0,11.0,3.0,2.0,3.0,2.0,2.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
X = X.drop(['Unnamed: 0'],axis=1)

In [12]:
X.to_csv("occurences_X.csv")

# preparer pred set

In [49]:
X_pred

0       Nous dûmes nous excuser des propos que nous eû...
1       Vous ne pouvez pas savoir le plaisir que j'ai ...
2       Et, paradoxalement, boire froid n'est pas la b...
3       Ce n'est pas étonnant, car c'est une saison my...
4       Le corps de Golo lui-même, d'une essence aussi...
                              ...                        
1195    C'est un phénomène qui trouve une accélération...
1196    Je vais parler au serveur et voir si on peut d...
1197    Il n'était pas comme tant de gens qui par pare...
1198        Ils deviennent dangereux pour notre économie.
1199    Son succès a généré beaucoup de réactions néga...
Name: sentence, Length: 1200, dtype: object

In [63]:
X_pred_occ = pd.read_csv('predSet_Occurences.csv')

In [64]:
X_pred_occ = X_pred_occ.drop(['Unnamed: 0'],axis=1)

In [ ]:
def preparation_predSet_occurences():
  prediction_set = pd.DataFrame(index=range(1200),columns=X.columns)
  i = 0
  for s in X_pred:
    c = Counter(get_wordTypes(s))
    print(c)
    print(i)
    for k in c.keys():
      if k in prediction_set.columns:
        prediction_set.iloc[i][k] = c[k]   
    i = i+1
  return prediction_set  


In [65]:
pred_set = preparation_predSet_occurences()

NameError: ignored

In [66]:
X_pred_occ = X_pred_occ.fillna(0)

In [67]:
X_pred_occ

,DET,NOUN,ADJ,VERB,ADP,PUNCT,CCONJ,ADV,PRON,AUX,SCONJ,PROPN,NUM,X,SYM,SPACE,INTJ
0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,1.0,2.0,3.0,1.0,1.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,3.0,0.0,0.0,3.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,2.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.0,13.0,9.0,7.0,11.0,6.0,5.0,8.0,11.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,5.0,7.0,1.0,3.0,3.0,2.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1196,1.0,2.0,0.0,5.0,1.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1197,9.0,14.0,6.0,12.0,16.0,6.0,2.0,4.0,14.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
1198,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pred_set.to_csv("predSet_Occurences.csv")

# REGRESSION

In [ ]:
classifier = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=4000, random_state=0)

In [ ]:
pipe = Pipeline([('classifier', classifier)])

In [ ]:
pipe.fit(X, y)

Pipeline(steps=[('classifier',
                 LogisticRegressionCV(cv=5, max_iter=2000, random_state=0))])

In [ ]:
predictions = pipe.predict(pred_set)

predictions = pd.DataFrame(predictions,columns=['difficulty'])

predictions.to_csv("LRCV_withNbOccurences2.csv")

#Now with TFID

In [22]:
X2 = df['sentence']
X2

0       Les coûts kilométriques réels peuvent diverger...
1       Le bleu, c'est ma couleur préférée mais je n'a...
2       Le test de niveau en français est sur le site ...
3                Est-ce que ton mari est aussi de Boston?
4       Dans les écoles de commerce, dans les couloirs...
                              ...                        
4795    C'est pourquoi, il décida de remplacer les hab...
4796    Il avait une de ces pâleurs splendides qui don...
4797    Et le premier samedi de chaque mois, venez ren...
4798    Les coûts liés à la journalisation n'étant pas...
4799    Sur le sable, la mer haletait de toute la resp...
Name: sentence, Length: 4800, dtype: object

In [23]:
tfidf = TfidfVectorizer(ngram_range=(1, 1))

ll = tfidf.fit_transform(X2)

In [24]:
xVect = pd.DataFrame(ll.todense(),
    columns=tfidf.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [68]:
oo = tfidf.transform(X_pred)

In [69]:
X_pred

0       Nous dûmes nous excuser des propos que nous eû...
1       Vous ne pouvez pas savoir le plaisir que j'ai ...
2       Et, paradoxalement, boire froid n'est pas la b...
3       Ce n'est pas étonnant, car c'est une saison my...
4       Le corps de Golo lui-même, d'une essence aussi...
                              ...                        
1195    C'est un phénomène qui trouve une accélération...
1196    Je vais parler au serveur et voir si on peut d...
1197    Il n'était pas comme tant de gens qui par pare...
1198        Ils deviennent dangereux pour notre économie.
1199    Son succès a généré beaucoup de réactions néga...
Name: sentence, Length: 1200, dtype: object

In [70]:
predVect = pd.DataFrame(oo.todense(),
    columns=tfidf.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [27]:
X_train = pd.concat([X,xVect],axis=1)

In [28]:
X_train

,DET,NOUN,ADJ,VERB,ADP,PUNCT,CCONJ,ADV,PRON,AUX,...,événement,événements,êtes,être,êtres,êut,île,îles,ôta,ôter
0,3.0,14.0,3.0,5.0,13.0,5.0,1.0,1.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,3.0,0.0,2.0,0.0,2.0,1.0,2.0,2.0,1.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,5.0,1.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,10.0,2.0,4.0,11.0,5.0,1.0,1.0,2.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,4.0,4.0,3.0,6.0,4.0,3.0,0.0,2.0,4.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4796,4.0,6.0,0.0,3.0,5.0,1.0,0.0,0.0,2.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4797,3.0,4.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4798,7.0,11.0,3.0,2.0,3.0,2.0,2.0,4.0,0.0,3.0,...,0.0,0.200821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
predVect

,000,02h00,03h00,10,100,1000,10000,105,11,110,...,événement,événements,êtes,être,êtres,êut,île,îles,ôta,ôter
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
X_pred = pd.concat([X_pred_occ,predVect],axis=1)

In [72]:
X_pred

,DET,NOUN,ADJ,VERB,ADP,PUNCT,CCONJ,ADV,PRON,AUX,...,événement,événements,êtes,être,êtres,êut,île,îles,ôta,ôter
0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,1.0,2.0,3.0,1.0,1.0,0.0,2.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,3.0,0.0,0.0,3.0,1.0,3.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,2.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.0,13.0,9.0,7.0,11.0,6.0,5.0,8.0,11.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,5.0,7.0,1.0,3.0,3.0,2.0,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1196,1.0,2.0,0.0,5.0,1.0,1.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1197,9.0,14.0,6.0,12.0,16.0,6.0,2.0,4.0,14.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1198,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
classifier = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=4000, random_state=0)

In [ ]:
pipe = Pipeline([('classifier', classifier)])

In [ ]:
pipe.fit(X_train, y)

In [ ]:
predictions = pipe.predict(X_pred)

predictions = pd.DataFrame(predictions,columns=['difficulty'])

predictions.to_csv("LRCV_withFeaturesBoosted.csv")